In [ ]:
class WrongAge(Exception):
    def __init__(self, message="Age should be greater than or equal to 0"):
        self.message = message
        super().__init__(self.message)

class AgeInvalid(Exception):
    def __init__(self, message="Son's age should be less than Father's age"):
        self.message = message
        super().__init__(self.message)

class Father:
    def __init__(self, age):
        if age < 0:
            raise WrongAge("Father's age cannot be negative")
        self.age = age

class Son(Father):
    def __init__(self, father_age, son_age):
        super().__init__(father_age)

        if son_age >= father_age:
            raise AgeInvalid("Son's age should be less than Father's age")

        self.son_age = son_age

# Example usage:
try:
    father = Father(-5)  # This will raise WrongAge exception
except WrongAge as e:
    print(e)

try:
    son = Son(40, 45)  # This will raise AgeInvalid exception
except AgeInvalid as e:
    print(e)


Father's age cannot be negative
Son's age should be less than Father's age


In [ ]:
class FormulaError(Exception):
    pass

def calculate(expression):
    try:
        elements = expression.split()
        if len(elements) != 3:
            raise FormulaError("Invalid input. Please provide a valid expression with 3 elements.")

        num1 = float(elements[0])
        operator = elements[1]
        num2 = float(elements[2])

        if operator not in ('+', '-'):
            raise FormulaError("Invalid operator. Please use '+' or '-'.")

        if operator == '+':
            result = num1 + num2
        else:
            result = num1 - num2

        print(f"Result: {result}")

    except ValueError:
        raise FormulaError("Invalid number. Please provide valid numerical values.")

# Main loop
while True:
    try:
        user_input = input("Enter a formula (or 'quit' to exit): ")
        if user_input.lower() == 'quit':
            break

        calculate(user_input)

    except FormulaError as e:
        print(f"Error: {e}")


In [ ]:
import threading
import random
import time

# Shared variable to store the random integer
shared_value = None

# Lock to synchronize access to the shared variable
lock = threading.Lock()

# Function for the first thread to generate a random integer every 1 second
def generate_random():
    global shared_value
    while True:
        with lock:
            shared_value = random.randint(1, 100)
        time.sleep(1)

# Function for the second thread to compute the square of the number (if even)
def compute_square():
    global shared_value
    while True:
        with lock:
            if shared_value is not None and shared_value % 2 == 0:
                square = shared_value ** 2
                print(f"Square of {shared_value}: {square}")
        time.sleep(1)

# Function for the third thread to print the cube of the number (if odd)
def print_cube():
    global shared_value
    while True:
        with lock:
            if shared_value is not None and shared_value % 2 != 0:
                cube = shared_value ** 3
                print(f"Cube of {shared_value}: {cube}")
        time.sleep(1)

# Create and start the threads
thread1 = threading.Thread(target=generate_random)
thread2 = threading.Thread(target=compute_square)
thread3 = threading.Thread(target=print_cube)

thread1.start()
thread2.start()
thread3.start()

# Keep the main thread alive
try:
    thread1.join()
    thread2.join()
    thread3.join()
except KeyboardInterrupt:
    print("Program terminated by user.")


Cube of 33: 35937
Cube of 47: 103823
Square of 82: 6724
Square of 66: 4356
Cube of 23: 12167
Square of 46: 2116
Square of 50: 2500
Square of 74: 5476
Square of 48: 2304
Square of 32: 1024
Program terminated by user.


In [ ]:
import threading
import queue
import time
import random

class VegetableMarket:
    def __init__(self, capacity):
        self.capacity = capacity
        self.market_queue = queue.Queue(maxsize=capacity)

    def add_vegetable(self, vegetable_type):
        if self.market_queue.full():
            print(f"Market is full. Farmer is waiting to add {vegetable_type}.")

        self.market_queue.put(vegetable_type)
        print(f"Farmer added {vegetable_type} to the market.")

    def sell_vegetable(self, consumer_name, vegetable_type):
        if self.market_queue.empty() or vegetable_type not in self.market_queue.queue:
            print(f"{consumer_name} is waiting for {vegetable_type} to be available.")

        vegetable = self.market_queue.get()
        print(f"{consumer_name} purchased {vegetable}.")

def farmer_thread(vegetable_market, vegetable_type):
    while True:
        time.sleep(random.uniform(1, 3))  # Simulating time to produce vegetables
        vegetable_market.add_vegetable(vegetable_type)

def consumer_thread(vegetable_market, consumer_name, vegetable_type):
    while True:
        time.sleep(random.uniform(1, 4))  # Simulating time for consumer to arrive
        vegetable_market.sell_vegetable(consumer_name, vegetable_type)

# Create Vegetable Market with capacity 5
market = VegetableMarket(capacity=5)

# Create farmer threads producing different vegetables
farmer1 = threading.Thread(target=farmer_thread, args=(market, "Tomatoes"))
farmer2 = threading.Thread(target=farmer_thread, args=(market, "Carrots"))

# Create consumer threads wanting to buy vegetables
consumer1 = threading.Thread(target=consumer_thread, args=(market, "Alice", "Tomatoes"))
consumer2 = threading.Thread(target=consumer_thread, args=(market, "Bob", "Carrots"))

# Start the threads
farmer1.start()
farmer2.start()
consumer1.start()
consumer2.start()

# Keep the main thread alive
try:
    farmer1.join()
    farmer2.join()
    consumer1.join()
    consumer2.join()
except KeyboardInterrupt:
    print("Program terminated by user.")

Cube of 49: 117649
Farmer added Tomatoes to the market.
Square of 94: 8836
Alice purchased Tomatoes.
Farmer added Carrots to the market.
Cube of 39: 59319
Cube of 85: 614125
Farmer added Tomatoes to the market.
Bob purchased Carrots.
Cube of 1: 1
Farmer added Carrots to the market.
Alice purchased Tomatoes.
Square of 30: 900
Farmer added Tomatoes to the market.
Square of 72: 5184
Bob purchased Carrots.
Alice purchased Tomatoes.
Farmer added Carrots to the market.
Square of 64: 4096
Alice is waiting for Tomatoes to be available.
Alice purchased Carrots.
Farmer added Tomatoes to the market.
Square of 30: 900
Farmer added Carrots to the market.
Square of 36: 1296
Bob purchased Tomatoes.
Farmer added Tomatoes to the market.
Program terminated by user.


In [ ]:
import threading
import time

# Synchronization using an Event
event = threading.Event()

# Function for Thread A to print numbers
def function_A():
    for num in range(1, 6):
        print(num, end=' ')
        event.set()  # Set the event to notify Thread B
        event.clear()  # Clear the event for the next iteration
        time.sleep(0.5)  # Simulate a time-consuming task

# Function for Thread B to print letters
def function_B():
    letters = ['A', 'B', 'C', 'D', 'E']
    for letter in letters:
        event.wait()  # Wait for the event from Thread A
        print(letter, end=' ')
        time.sleep(0.5)  # Simulate a time-consuming task

# Create threads
thread_A = threading.Thread(target=function_A)
thread_B = threading.Thread(target=function_B)

# Start both threads
thread_A.start()
thread_B.start()

# Wait for both threads to complete
thread_A.join()
thread_B.join()

# Print a newline to separate the output from the next prompt
print()

1 2 A 3 4 B 5 

KeyboardInterrupt: 